# Installing required libraries


Libraries used:

1. simple_image_download - it returns the urls derived from the google image search of the fashion keywords.

2. flask - it helps us to host our UI 

3. textrazor - an NLP API for keyword extraction

4. flair - NLP Framework for POS tagging

5. pytrends - an API for analysing data from Google Trends

In [ ]:
!pip install git+https://github.com/RiddlerQ/simple_image_download.git@2eb34f88bd275723809193b5d9349866cca048aa

In [ ]:
!pip install Flask-SocketIO
!pip install textrazor
!pip install pytrends
!pip install flair
!pip install flask-ngrok
!pip install flask==0.12.4

In [ ]:
pip install Werkzeug==0.16.1

# Importing the required libraries

In [ ]:
from flask import Flask, flash, redirect, render_template, request, session, abort , url_for,jsonify,Response,session,request,g,make_response
from flask_socketio import SocketIO
import time,glob,random,sqlite3,os,requests
from flask_ngrok import run_with_ngrok
import requests
from bs4 import BeautifulSoup,SoupStrainer
import httplib2
from googlesearch import search 
import os
import unicodedata
import re
import textrazor
from pytrends.request import TrendReq
import pandas as pd
import time
import csv
from datetime import datetime
from flair.data import Sentence
from flair.models import SequenceTagger
from simple_image_download import simple_image_download as simp

# Code

Mounting Google Drive to our session storage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/
!mkdir GRID
%cd /content/drive/My\ Drive/GRID

Declaring the global variables

In [ ]:
url_best = []
best_name=[]
s=''
g=''
p=''
url_t=[]
url_l=[]
url_n=[]
bug=[]
keyword_t=[]
keyword_l=[]
nodata=[]
final_tags=[]
final_entities=[]
final_images_dir=[]

## Code to run webpage


In [ ]:
app = Flask(__name__)
run_with_ngrok(app)
app.secret_key=os.urandom(24)
socketio = SocketIO(app)

#code for the landing page
@app.route('/',methods=['POST','GET'])
def login():
	global s
	global g
	global p
	if(request.form.get('type')=='Trend'):
		return render_template('trending.html',site=s,garment=g,url_trend=url_t,keyword_trend=keyword_t,len=len(url_t))
	elif(request.form.get('type')=='Lag'):
		return render_template('lagging.html',site=s,garment=g,url_lag=url_l,keyword_lag=keyword_l,len=len(url_l),len_nodata=len(url_n),url_nodata=url_n,keyword_nodata=nodata)
	elif(request.form.get('type')=='Best'):
		return render_template('best_sellers.html',site=s,garment=g,url_best=url_best,best_seller=best_name,len=len(url_best))
	elif(request.form.get('type')=='Upcoming'):
		return render_template('portals.html',portal=p,garment=g,len=len(final_images_dir),images=final_images_dir,filters=final_tags,entities=final_entities,len1=bug)
	if(request.form.get('site')!=None):
		s=request.form.get('site')
		g=request.form.get('garment')
		p=request.form.get('portal')
		search(g,s)
		return render_template('overview.html',site=s,portal=p,garment=g,url_trend=url_t,url_lag=url_l,best=url_best,images=final_images_dir)   
	return render_template('fashion.html')
 
#code for the t-shirt simulator
@app.route("/simulator/", methods=['POST'])
def move_forward():
		forward_message = "Moving Forward..."
		return render_template('simulator.html', forward_message=forward_message);

#code for the chatbot
@app.route('/getd/<data>')
def getd(data):
        result=''	
        l=[]
        page=data	
        api_key = "a9170a3ab84748cc681f9fe929b19808"
        name=str(page)
        x=name.split()
        aa=x[0]
        age=int(x[1])	
        base_url = "http://api.openweathermap.org/data/2.5/weather?"
        city_name=aa
  
# complete_url variable to store 
# complete url address 
        complete_url = base_url + "appid=" + api_key + "&q=" + city_name 
      
    # get method of requests module 
    # return response object 
        response = requests.get(complete_url) 
        x = response.json() 
      
    # Now x contains list of nested dictionaries 
    # Check the value of "cod" key is equal to 
    # "404", means city is found otherwise, 
    # city is not found 
        if x["cod"] != "404": 
      
        # store the value of "main" 
        # key in variable y 
             y = x["main"] 
      
        # store the value corresponding 
        # to the "temp" key of y 
             current_temperature = y["temp"] 
      
        # store the value corresponding 
        # to the "pressure" key of y 
             current_pressure = y["pressure"] 
      
        # store the value corresponding 
        # to the "humidity" key of y 
             current_humidiy = y["humidity"] 
      
        # store the value of "weather" 
        # key in variable z 
             z = x["weather"] 
      
        # store the value corresponding  
        # to the "description" key at  
        # the 0th index of z 
             weather_description = z[0]["description"] 
             l=[str(current_temperature-273),str(current_pressure),str(current_humidiy), str(weather_description)]
        # print following values 
             current_temperature-=273
        if(len(l)==0):
                result="City not found"

        elif(current_temperature<10):
                result="Farishta suggests you to design woollen sweaters or jackets of "
                if(age<=15):
                        result+="bright colours."
                else:
                        result+="dark colours."	
        elif(current_temperature>=20 and age<=12):
             result="Farishta suggests you to design short sleeve t-shirts of light colour such as white,yellow, etc and prints of cartoons characters like Chhota Bheem, Doraemon, Spiderman etc "
        elif(current_temperature>=20 and age>12 and age<=18):
             result="Farishta suggests you to design cotton short sleeve t-shirts of light colour such as white,yellow, etc and prints of sports, sportsman, education, science etc."
        elif(current_temperature>=20and age>18 and age<=30):
             result="Farishta suggests you to design cotton short sleeve casual t-shirts."
        elif(current_temperature>=20and age>30 and age<=50):
             result="Farishta suggests you to design cotton short sleeve solid light colour t-shirts."		

        elif(current_temperature<20 and age<=12):
             result="Farishta suggests you to design full sleeve t-shirts of dark colour such as black, brown, etc and cartoons characters like Chhota Bheem, Doraemon, Spiderman etc "
        elif(current_temperature<20 and age>12 and age<18):
             result="Farishta suggests you to design short sleeve t-shirts of light colour such as white,yellow, etc and prints of sports, sportsman, education, science etc."
        elif(current_temperature<20 and age>=18 and age<=30):
             result="Farishta suggests you to design hoodies with fictional characters on it."
        elif(current_temperature<20 and age>30 and age<=50):
             result="Farishta suggests you to design full sleeve solid dark colour t-shirts with some quotes."
        else:
             result="Are you comedy me?\n"
        result+="\nFarishta suggests you please checkout our T-shirt simulator feature. Its still a beta product will make it final soon."
        return jsonify(r=result)

# Getting images from query sites


 Once the trending and lagging attributes are understood, we make a google search of those attributes to obtain images to serve the user.

In [ ]:
def search(query,brand):
    c=0
    fashion=''
    global trending
    global lagging
    trending={}
    lagging={}
    global url_t
    global url_l
    global url_n
    global nodata
    url_t=[]
    url_l=[]
    url_n=[]
    nodata=[]
    global final_tags
    global final_entities
    global final_images_dir
    final_tags=[]
    final_entities=[]
    final_images_dir=[]
    try: 
    	from googlesearch import search 
    except ImportError: 
    	print("No module named 'google' found") 
    
    #the query is processed according to the site chosen
    for j in search(query+" "+brand, tld="co.in", num=10, stop=10, pause=2): 
        if(query.casefold() in j or brand.casefold() in j):
          if(s.casefold()=='koovs'):
            koovs(j)
          elif(s.casefold()=='paytm-mall'):
            paytm(j)
          break

# Getting trending and lagging attributes from e-commerce sites.

We utilise the DOM structure of the respective website to obtain information required to predict trending and lagging features.

In [ ]:
def koovs(a):
    global c
    http = httplib2.Http()
    status, response = http.request(a)
    soup=BeautifulSoup(response,'lxml' )
    colors=""
    colo=[]
    brand=[]
    material=[]
    others=[]
    spare=[]
    koovs_trend=[]
    desc=[]
    for lnk in BeautifulSoup(response, 'html.parser',parse_only=SoupStrainer('img')):
      if(lnk.has_attr('src') and 'product' in lnk['src'] and 'jpg' in lnk['src'] and lnk.has_attr('alt')):    
        koovs_trend.append(lnk['src'])
        desc.append(lnk['alt'])
    #due to the order in which images load during scraping, we pick the appropriate ones
    koovs_trend=koovs_trend[2:12]
    desc=desc[2:12]
    global url_best
    global best_name
    url_best=koovs_trend
    best_name=desc

    for link in BeautifulSoup(response, 'html.parser',parse_only=SoupStrainer('a')):
        if (link.has_attr('href') and 'html' in link['href'] ) :
            s="https://www.koovs.com"+link['href']
            status1, response1 = http.request(s)
            for spa in BeautifulSoup(response1,'html.parser',parse_only=SoupStrainer(class_='breadcrumb')):
                colors+=(spa.text+" ")
            for sp in BeautifulSoup(response1,'html.parser',parse_only=SoupStrainer(class_='desc')):
                clean_text = unicodedata.normalize("NFKD",sp.text)
                spare.append(clean_text.split('\n'))
               
    #we utilise specifics of site to extract info from the site
    for i in spare:
        if(' ' in i):
            i.remove(' ')
        i.remove('')
        brand.append(i[0][(i[0].index('by')+2):].strip())
        line = i[1]
        matchObj = re.match( r'Made from (.*)', line, re.M|re.I)

        if matchObj:
          material.append( matchObj.group(1).strip())

        else:
            continue

        for j in range(3,len(i)):
            if(i[j]!=''):
                others.append(i[j].strip())
        

    colors=colors.split()
    colors1=['alice blue', 'antique white', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanched almond', 'blue', 'blue violet', 'brown', 'burly wood', 'cadet blue', 'chartreuse', 'chocolate', 'coral', 'cornflower blue', 'cornsilk', 'crimson', 'cyan', 'dark blue', 'dark cyan', 'dark golden rod', 'dark gray', 'dark grey', 'dark green', 'dark khaki', 'dark magenta', 'dark olive green', 'dark orange', 'dark orchid', 'dark red', 'dark salmon', 'dark sea green', 'dark slate blue', 'dark slate gray', 'dark slate grey', 'dark turquoise', 'dark violet', 'deep pink', 'deep sky blue', 'dim gray', 'dim grey', 'dodger blue', 'fire brick', 'floral white', 'forest green', 'fuchsia', 'gainsboro', 'ghost white', 'gold', 'golden rod', 'gray', 'grey', 'green', 'green yellow', 'honey dew', 'hot pink', 'indian red', 'indigo', 'ivory', 'khaki', 'lavender', 'lavender blush', 'lawn green', 'lemon chiffon', 'light blue', 'light coral', 'light cyan', 'light golden rod yellow', 'light gray', 'light grey', 'light green', 'light pink', 'light salmon', 'light sea green', 'light sky blue', 'light slate gray', 'light slate grey', 'light steel blue', 'light yellow', 'lime', 'lime green', 'linen', 'magenta', 'maroon', 'medium aqua marine', 'medium blue', 'medium orchid', 'medium purple', 'medium sea green', 'medium slate blue', 'medium spring green', 'medium turquoise', 'medium violet red', 'midnight blue', 'mint cream', 'misty rose', 'moccasin', 'navajo white', 'navy', 'old lace', 'olive', 'olive drab', 'orange', 'orange red', 'orchid', 'pale golden rod', 'pale green', 'pale turquoise', 'pale violet red', 'papaya whip', 'peach puff', 'peru', 'pink', 'plum', 'powder blue', 'purple', 'rebecca purple', 'red', 'rosy brown', 'royal blue', 'saddle brown', 'salmon', 'sandy brown', 'sea green', 'sea shell', 'sienna', 'silver', 'sky blue', 'slate blue', 'slate gray', 'slate grey', 'snow', 'spring green', 'steel blue', 'tan', 'teal', 'thistle', 'tomato', 'turquoise', 'violet', 'wheat', 'white', 'white smoke', 'yellow', 'yellow green']
    for i in (colors):
        
        if(i.casefold() in colors1):
            colo.append(i)

    coloring=set(colo) 
     
    uw=set(others)
    uw1=set(brand)
    uw2=set(material)
    brand_count=[]

    duw1={}
    duw2={}
    duw3={}
    duw4={}
    for i in uw1:
        duw1[i]=brand.count(i)
    for i in uw:
        duw2[i]=others.count(i)
    for i in uw2:
        duw3[i]=material.count(i)
    for i in coloring:
        duw4[i]=colo.count(i)
    process(duw1,duw2,duw3,duw4)


#similarly, the specifics of this site are used to extract our required information
def paytm(a):
    
    global c
    http = httplib2.Http()
    status, response = http.request(a)
    soup=BeautifulSoup(response,'lxml' )
    colors=""
    types=['Product Type','Brand','Product Code','Color','Size','Material','Occasion','Length','Pattern','Sleeve','Neck Type','Fit','Gender','Hood','Set Contents','Wash Care','Disclaimer','Pattern Coverage','Combo Size']
    colo=[]
    brand=[]
    material=[]
    others=[]
    spare=[]
    koovs_trend=[]
    desc=[]
    coloring=[]
    size=['S','M','L','XL']
    for lnk in BeautifulSoup(response, 'html.parser',parse_only=SoupStrainer('img')):
        if(lnk.has_attr('src') and 'product' in lnk['src'] and 'jpg' in lnk['src'] and lnk.has_attr('alt')):
            
            koovs_trend.append(lnk['src'])
            desc.append(lnk['alt'])
    koovs_trend=koovs_trend[2:12]
    desc=desc[2:12]
    global url_best
    global best_name
    url_best=koovs_trend
    best_name=desc

    for link in BeautifulSoup(response, 'html.parser',parse_only=SoupStrainer(class_='_8vVO')):
   
        if (link.has_attr('href')   and 'http' not in link['href'] ) :
        
            s='https://paytmmall.com'+link['href']
            
            status1, response1 = http.request(s)
            others={}
            for (sp,typ) in zip(BeautifulSoup(response1,'html.parser',parse_only=SoupStrainer(class_='_2LOI')),types):
                others[typ]=sp.text
            colo.append(others['Color'])
            brand.append(others['Brand'])
            if(others['Material'] not in size ):
                material.append(others['Material'])
            spare.append(others['Occasion'])
            spare.append(others['Length'])
            spare.append(others['Pattern'])
            spare.append(others['Sleeve'])
            spare.append(others['Neck Type'])
            spare.append(others['Fit'])
               
    colors1=['alice blue', 'antique white', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanched almond', 'blue', 'blue violet', 'brown', 'burly wood', 'cadet blue', 'chartreuse', 'chocolate', 'coral', 'cornflower blue', 'cornsilk', 'crimson', 'cyan', 'dark blue', 'dark cyan', 'dark golden rod', 'dark gray', 'dark grey', 'dark green', 'dark khaki', 'dark magenta', 'dark olive green', 'dark orange', 'dark orchid', 'dark red', 'dark salmon', 'dark sea green', 'dark slate blue', 'dark slate gray', 'dark slate grey', 'dark turquoise', 'dark violet', 'deep pink', 'deep sky blue', 'dim gray', 'dim grey', 'dodger blue', 'fire brick', 'floral white', 'forest green', 'fuchsia', 'gainsboro', 'ghost white', 'gold', 'golden rod', 'gray', 'grey', 'green', 'green yellow', 'honey dew', 'hot pink', 'indian red', 'indigo', 'ivory', 'khaki', 'lavender', 'lavender blush', 'lawn green', 'lemon chiffon', 'light blue', 'light coral', 'light cyan', 'light golden rod yellow', 'light gray', 'light grey', 'light green', 'light pink', 'light salmon', 'light sea green', 'light sky blue', 'light slate gray', 'light slate grey', 'light steel blue', 'light yellow', 'lime', 'lime green', 'linen', 'magenta', 'maroon', 'medium aqua marine', 'medium blue', 'medium orchid', 'medium purple', 'medium sea green', 'medium slate blue', 'medium spring green', 'medium turquoise', 'medium violet red', 'midnight blue', 'mint cream', 'misty rose', 'moccasin', 'navajo white', 'navy', 'old lace', 'olive', 'olive drab', 'orange', 'orange red', 'orchid', 'pale golden rod', 'pale green', 'pale turquoise', 'pale violet red', 'papaya whip', 'peach puff', 'peru', 'pink', 'plum', 'powder blue', 'purple', 'rebecca purple', 'red', 'rosy brown', 'royal blue', 'saddle brown', 'salmon', 'sandy brown', 'sea green', 'sea shell', 'sienna', 'silver', 'sky blue', 'slate blue', 'slate gray', 'slate grey', 'snow', 'spring green', 'steel blue', 'tan', 'teal', 'thistle', 'tomato', 'turquoise', 'violet', 'wheat', 'white', 'white smoke', 'yellow', 'yellow green']
    for i in (colo):
        
        if(i.casefold() in colors1):
            coloring.append(i)
    colors=set(coloring)       
    uw1=set(brand)
    uw2=set(material)
    
    uw4=set(spare)
    brand_count=[]
    duw1={}
    duw2={}
    duw3={}
    duw4={}
    for i in uw1:
        duw1[i]=brand.count(i)
    for i in uw2:
        duw2[i]=material.count(i)
    for i in colors:
        duw3[i]=coloring.count(i)
    for i in uw4:
        duw4[i]=spare.count(i)
    process(duw1,duw4,duw2,duw3)


# Extracting the required trending and lagging features.

We use keyword extraction methods using filters to extract fashion keywords from the information that was scraped.

In [ ]:
 def process(duw1,duw2,duw3,duw4):
    duw1={k: v for k, v in sorted(duw1.items(), key=lambda item: item[1],reverse=True)}
    duw2={k: v for k, v in sorted(duw2.items(), key=lambda item: item[1],reverse=True)[:10]}
    duw3={k: v for k, v in sorted(duw3.items(), key=lambda item: item[1],reverse=True)}
    duw4={k: v for k, v in sorted(duw4.items(), key=lambda item: item[1],reverse=True)}
    #keyword extraction methods to get fashion attributes
    for i in duw2.keys():
      client = textrazor.TextRazor(api_key="ef90a8665ca910ec35ee078d2414232e08eb3b393d160d05f7b738bf", extractors=["words","phrases"])
      client.set_entity_freebase_type_filters(["/fashion/fashion_category", "/fashion/fashion_designer","/fashion/fashion_label","/fashion/fashion_week","/fashion/fiber","/fashion/garment","/fashion/textile","/fashion/weave"])
      to_analyze = i
      response = client.analyze(to_analyze)
      keywords = []
      for np in response.noun_phrases():
        keyword = to_analyze[np.words[0].input_start_offset: np.words[-1].input_end_offset]
        keywords.append(keyword)
      if len(keywords) >1:
        keywords1 = keywords
        for j in keywords1:
          sentence = Sentence(j)
          #utilisation of POS tagging
          tagger = SequenceTagger.load('pos')
          tagger.predict(sentence)
          for entity in sentence.get_spans('pos'):
            if (str(entity).split()[5]) == 'DT':
              keywords.remove(j)
      if(len(keywords)!=0):
        csv_read(keywords[0])
      else:
        csv_read(i)
    trend={k: v for k, v in sorted(trending.items(), key=lambda item: item[1])}
    lag={k: v for k, v in sorted(lagging.items(), key=lambda item: item[1])}
    garment=g
    trend_search=[]
    lag_search=[]
    a=b=c=0
    for i in trend:
      keyword_t.append([list(duw4.keys())[a],list(duw1.keys())[b],list(duw3.keys())[c],i])
      trend_search.append(list(duw4.keys())[a]+" "+list(duw1.keys())[b].casefold()+" "+list(duw3.keys())[c]+" "+i+" "+garment)
      a=a+1
      if(a==len(duw4)): a=a-1
      b=b+1
      if(b==len(duw1)): b=b-1
      c=c+1
      if(c==len(duw3)): c=c-1
    print(trend_search)
    a=len(duw4)-1
    b=len(duw1)-1
    c=len(duw3)-1
    for i in lag:
      keyword_l.append([list(duw4.keys())[a],list(duw1.keys())[b],list(duw3.keys())[c],i])
      lag_search.append(list(duw4.keys())[a]+" "+list(duw1.keys())[b].casefold()+" "+list(duw3.keys())[c]+" "+i+" "+garment)
      a=a-1
      if(a==-1): a=0
      b=b-1
      if(b==-1): b=0
      c=c-1
      if(c==-1): c=0
    print(lag_search)
    print(nodata)
    if(len(trend_search)!=0):
      download(trend_search,6,1)
    if(len(lag_search)!=0):
      download(lag_search,6,2)
    if(len(nodata)!=0):
      download(nodata,2,3)
    mega(g, p)

The statistical search-related data of the keywords are stored as a csv file using Google Trends.

In [ ]:
 def csv_read(string):
  df2 = ['']
  flag = True
  startTime = time.time()
  pytrend = TrendReq(hl='en-GB', tz=360)
  df2[0]=string
  dataset = []
  for x in range(0,len(df2)):
      keywords = [df2[x]]
      pytrend.build_payload(
      kw_list=keywords,
      cat=68,
      timeframe='2014-12-31 2020-03-01')
      #we set a timeframe of past 5 years approx to get an idea of seasonal trends for each product/keyword
      #different products/keywords have different periods in which they trend/lag and we extract these patterns
      data = pytrend.interest_over_time()
      if not data.empty:
            data = data.drop(labels=['isPartial'],axis='columns')
            dataset.append(data)
  try:
    result = pd.concat(dataset, axis=1)
  except:
    global nodata
    nodata.append(string)
    flag = False 
  if (flag == True):
    result.to_csv('/content/GRID/'+'trends.csv')
    predict()

We utilise the power of Google Trends which is freely available to predict which features are trending and lagging.


To do this, we analyze trends for each of the keyword during past years to see if it currently trending or lagging.

Utilising past trends allows us to avoid unicorns, which are sudden spike trends which die out soon. 

In [ ]:
def predict():
  maxima={}
  #with the patterns extracted from previous function, we find the times during which the sale increases and the times during which sales fall
  #accordingly, we suggest if its desirable or undesirable to sell the product at this time of the year
  with open('/content/GRID/trends.csv', 'r') as file:
      reader = csv.reader(file)
      i=-1
      year = 2015
      max = ['',-1]
      for row in reader:
          i=i+1
          if(i<1):
            fashion=row[1]
            continue
          if(year==2020): break
          if(row[0].startswith(str(year))):
              if(int(row[1])>max[1]): 
                  max[1]=int(row[1])
                  max[0]=row[0]
          else:
              year=year+1
              maxima[max[0]]=max[1]
              max[1] = -1
              max[0] = ''
              if(int(row[1])>max[1]): 
                  max[1]=int(row[1])
                  max[0]=row[0]
  with open('/content/GRID/trends.csv', 'r') as file:
      reader = csv.reader(file)
      min = ['',101]
      i=-1
      num=0;
      minima = {}
      flag = False
      keys = list(maxima.keys())
      for row in reader:
          i=i+1
          if(i<1): continue
          if(row[0]==keys[num]):
              num=num+1
              if(flag):
                  minima[min[0]]=min[1]
                  min[1] = 101
                  min[0] = ''
              if(num==5): break
              flag = True
          elif(flag and row[0]!=keys[0]):
              if(int(row[1])<min[1]):
                  min[1]=int(row[1])
                  min[0]=row[0]
  max_month=0
  min_month=0
  for x in maxima:
      max_month=max_month+int(list(x.split("-"))[1])
  max_month=max_month/5
  for x in minima:
      min_month=min_month+int(list(x.split("-"))[1])
  min_month=min_month/4
  today = datetime.today()
  datem = datetime(today.year, today.month, 1)
  s=str(datem)[5:7]
  if(s[0]=='0'):
    month=int(s[1])
  else:
    month=int(s)
  if(max_month>min_month):
      if(month<=min_month): 
        lagging[fashion]=float('{:.2f}'.format(min_month-month))
      elif(month>min_month and month<=max_month): 
        trending[fashion]=float('{:.2f}'.format(max_month-month))
      else: 
        lagging[fashion]=float('{:.2f}'.format(min_month+(12-month)))
  else:
      if(month<=max_month): 
        trending[fashion]=float('{:.2f}'.format(max_month-month))
      elif(month>max_month and month<=min_month): 
        lagging[fashion]=float('{:.2f}'.format(min_month-month))
      else: 
        trending[fashion]=float('{:.2f}'.format(max_month+(12-month)))

# Downloading the images of trending and lagging feature products.

Once we identify the trending and lagging features, we are no longer constrained by the site to obtain pictures. We then proceed to obtain pictures for the products from Google search. These will be shown to the user.

In [ ]:
def download(pics,n,num):
  for i in pics:
    response = simp.simple_image_download
    if(num==1):url_t.append(response().urls(i,n))
    elif(num==2):url_l.append(response().urls(i,n))
    else:url_n.append(response().urls(i,n))

# Fashion Portals

We enter the respective fashion portal and using keyword extraction to filter out the articles that contain our garment of interest. Once that is done, we proceed to obtain insights from our articles of interest. Entites like models, colors of garments, style of fashion and garments, etc are obtained. This can give a lot of insights to the the designer, as in which models are trending right now, the trending color, key locations where fashion is booming, etc. The model can even detect references to tv-shows, fictional characters, famous figures, cartoons, etc. Since fashion is influenced by a lot of features, we leave almost no stone unturned as in identifying  key factors driving fashion.

In [ ]:
def mega(garment, site):

  c=0
  global final_entities
  global final_images_dir
  global final_tags
  corpus_list=[]

#since scraping code is not one size fits all, we choose the scraping code depending on the site.

  def vogue():
    
      a="https://www.vogue.co.uk/fashion/fashion-trends"
      global c
      http = httplib2.Http()
      status, response = http.request(a)
      soup=BeautifulSoup(response,'lxml' )

      for link in BeautifulSoup(response, 'html.parser',parse_only=SoupStrainer('a')):

          k=""
          if (link.has_attr('href') and 'article' in link['href'] ) :
              s="https://www.vogue.co.uk"+link['href']
              k=k+s+'\n'
              status1, response1 = http.request(s)
              for spa in BeautifulSoup(response1,'html.parser',parse_only=SoupStrainer('p')):
                  k=k+spa.text
              corpus_list.append(k)

  def elle():
    a="https://www.elle.com/fashion/trend-reports/"
    global c
    http = httplib2.Http()
    status, response = http.request(a)
    soup=BeautifulSoup(response,'lxml' )
    for link in BeautifulSoup(response, 'html.parser',parse_only=SoupStrainer('a')):

        k=""
        if (link.has_attr('href') and 'trend-reports' in link['href'] and 'http' not in link['href'] ) :
            s="https://www.elle.com"+link['href']
            k=k+s+'\n'
            status1, response1 = http.request(s)
            for spa in BeautifulSoup(response1,'html.parser',parse_only=SoupStrainer('p')):
                k=k+spa.text
            corpus_list.append(k)

  def esquire():
    
      a="https://www.esquire.com/style/mens-fashion/"
      global c
      http = httplib2.Http()
      status, response = http.request(a)
      soup=BeautifulSoup(response,'lxml' )

      for link in BeautifulSoup(response, 'html.parser',parse_only=SoupStrainer('a')):

          k=""
          if (link.has_attr('href') and 'style/mens-fashion' in link['href'] and 'http' not in link['href']) :
              #print(link['href'])
              s="https://www.esquire.com/"+link['href']
              k=k+s+'\n'
              status1, response1 = http.request(s)
              for spa in BeautifulSoup(response1,'html.parser',parse_only=SoupStrainer('p')):
                  k=k+spa.text
              corpus_list.append(k)

  if p.casefold() == 'vogue':
    vogue()

  elif p .casefold() == 'esquire':
    esquire()

#since magazine titles are often misleading, we use keyword extraction to understand which garment the article deals with
  def detect_target_garment(content, target):
    client = textrazor.TextRazor(api_key="13b22cd6d8562948feeddee54a992ef4edfb1b9d8c3df54a70f40810", extractors=["entities"])
    client.set_entity_freebase_type_filters(["/fashion/garment", "/business/product_category"])
    to_analyze = content
    response = client.analyze(to_analyze)
    garment = []
    for ent in response.entities():
      garment.append(ent.id)
    if target in garment:
      return True

#we filter out and retain only the articles of our interest
  def  flush(corpus_list, target_garment):
    refined_list = []
    for i, article in enumerate(corpus_list[:-1]):
      url = article.partition('\n')[0]
      content = article
      res = detect_target_garment(content,target_garment)
      if res == True:
        refined_list.append(content)    
    return refined_list

  target = garment
  refined_list = flush(corpus_list, target)

  refined_urls = []
  for i in refined_list:
    refined_urls.append(i.partition('\n')[0])

#code is tweaked to extract pics from respective site of our interest
  def refined_url_pics(url):
    images_list = []
    a=url
    global c
    http = httplib2.Http()
    status, response = http.request(a)
    soup=BeautifulSoup(response,'lxml' )
    for link in BeautifulSoup(response, 'html.parser',parse_only=SoupStrainer('img')):

        if (link.has_attr('srcset') and 'vogue' in url  ) :
            l=(link['srcset'].split(','))
            images_list.append(l[len(l)-1][:-6])
        if (link.has_attr('data-src') and link.has_attr('alt') and link['alt']!='' and 'hips' in link['data-src'] and 'esquire' in url) :
            images_list.append(link['data-src'])
            break     
            
    return images_list

  for url in refined_urls:
    final_images_dir.append(refined_url_pics(url))

  print(final_images_dir)

#we use a huge list of filters to identify useful insights from the articles of our interest
#the model we use is powerful enough to understand complex context as well
#entities like model names, locations, fashion styles are picked up to keep the user informed of the current meta
  filter_list = ['/fashion/fashion_category', '/fashion/fashion_label', '/fashion/fashion_week', '/fashion/fashion_designer', '/fashion/fiber', '/fashion/textile', '/fashion/weave', '/business/industry', '/business/product_category', '/business/product_line', '/business/sponsor', '/celebrities/celebrity', '/celebrities/sexual_orientation', '/celebrities/supercouple', '/comic_books/comic_book_character', '/comic_books/comic_book_fictional_universe', '/comic_strips/comic_strip', '/comic_strips/comic_strip_character', '/computer/computer_designer', '/cricket/cricket_player', '/exhibitions/exhibit', '/exhibitions/exhibition', '/fictional_universe/fictional_character', '/fictional_universe/fictional_object', '/fictional_universe/fictional_universe', '/fictional_universe/person_in_fiction', '/fictional_universe/work_of_fiction', '/film/film_character', '/film/film_genre', '/film/person_or_entity_appearing_in_film', '/internet/blog', '/internet/social_network_user', '/internet/website', '/location/citytown', '/location/country', '/location/region', '/media_common/creative_work', '/media_common/dedicated_work', '/media_common/finished_work', '/media_common/netflix_title', '/music/artist', '/music/festival', '/music/music_video_character', '/music/musical_group', '/music/producer', '/music/track', '/music/soundtrack', '/music/single', '/music/music_video', '/music/genre', '/organization/australian_organization', '/organization/club', '/organization/club_interest', '/organization/endowed_organization', '/organization/membership_organization', '/organization/non_profit_designation', '/organization/non_profit_organization', '/organization/organization', '/organization/organization_committee', '/organization/organization_committee_title',    '/organization/organization_type', '/tv/tv_genre', '/tv/tv_personality']

  def tag_extractor(article):
    client = textrazor.TextRazor(api_key="13b22cd6d8562948feeddee54a992ef4edfb1b9d8c3df54a70f40810", extractors=["entities"])
    client.set_entity_freebase_type_filters(filter_list)
    to_analyze = article
    response = client.analyze(to_analyze)
    tags_dir = {}
    for ent in response.entities():
      intersection = list(set(ent.freebase_types) & set(filter_list))
      for i in intersection:
        k = i[1:].upper()
        if k  in tags_dir.keys():
          tags_dir[k].append(ent.id)
        else:
          tags_dir[k] = [ent.id]
    tags =[]
    entities = []
    
    for i in tags_dir.keys():
      entities.append(tags_dir[i])
      tags.append(i)
    return tags, entities

  for i in refined_list:
    a, b = tag_extractor(i)

    final_tags.append(a)
    final_entities.append(b)
  len1=[]
  for i in final_tags:
    len1.append(len(i))
  global bug
  bug=len1
  for i,x in enumerate(final_entities):
    for j,y in enumerate(x):
        final_entities[i][j] = list(set(y))
  print(final_tags)
  print(final_entities)

#driver code to trigger our website
if __name__ == "__main__":
    app.run()

2020-08-09 12:48:34,396 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt
2020-08-09 12:48:35,011 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt
['Black koovs cotton Muscle fit T-shirt', 'Blue chelsea king polyester blend Regular fit T-shirt', 'Green adidas originals polyester Long sleeves T-shirt', 'Pink adamo london cotton blend Slim fit T-shirt', 'Grey fila poly-spandex blend Half placket T-shirt']
['Yellow k denim polyester-spandex blend Short sleeves T-shirt']
['Side-split hem', 'Half-placket button closure', 'Half placket button closure', 'Contrast shoulder tape']
[[' https://media.vogue.co.uk/photos/5eecb6815f6672410c7cdb85/2:3/w_3840%2cc_limit/GettyImages-1037457514.jpg', ' https://media.vogue.co.uk/photos/5e981bf8eb104e0008b35453/3:2/w_3840%2cc_limit/GettyImages-1133271785.jpg', ' https://media.vogue.co.uk/photos/5eecb67c5f6672410c7cdb81/master/w_3840%2cc_limit/GettyImages-1175606107.jpg', ' https://media.vogue.co.uk/photos/5eecb67f8d5f1419f3619883/master/w_

127.0.0.1 - - [09/Aug/2020 12:49:03] "POST / HTTP/1.1" 200 -


[['LOCATION/COUNTRY', 'BUSINESS/PRODUCT_CATEGORY', 'BUSINESS/INDUSTRY', 'FICTIONAL_UNIVERSE/PERSON_IN_FICTION', 'MUSIC/ARTIST', 'FILM/PERSON_OR_ENTITY_APPEARING_IN_FILM', 'CELEBRITIES/CELEBRITY', 'TV/TV_PERSONALITY', 'ORGANIZATION/ORGANIZATION', 'TV/TV_GENRE', 'FASHION/FASHION_LABEL', 'FILM/FILM_GENRE', 'ORGANIZATION/ORGANIZATION_TYPE', 'MEDIA_COMMON/NETFLIX_TITLE', 'FASHION/TEXTILE']]
[[['United Kingdom', 'United States'], ['Blazer', 'Jacket', 'Leather jacket', 'T-shirt'], ['Fashion', 'T-shirt'], ['James Dean', 'Brooke Shields'], ['Christy Turlington', 'James Dean', 'Brooke Shields'], ['Cindy Crawford', 'James Dean', 'Christy Turlington', 'Brooke Shields'], ['Cindy Crawford', 'Christy Turlington', 'Brooke Shields'], ['Cindy Crawford'], ['Condé Nast', 'Cindy Crawford', 'Fashion'], ['Fashion'], ['Fashion'], ['Fashion'], ['Fashion'], ['Rebel Without a Cause'], ['Denim']]]


127.0.0.1 - - [09/Aug/2020 12:51:12] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2020 12:51:35] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2020 12:51:58] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2020 12:52:12] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2020 12:52:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2020 12:53:22] "GET /getd/Auckland%2030 HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2020 12:53:45] "GET /getd/Mumbai%2020 HTTP/1.1" 200 -
[2020-08-09 12:53:59,552] ERROR in app: Exception on /getd/Bye [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist

['Black nike Classic Canvas Sneakers', 'White foonike-men-s-tshak29399de787644 Athleisure Synthetic Sneakers', 'Blue foonike-men-s-tshak29399de787644 Slip-On Round Toe Sneakers', 'Grey foonike-men-s-tshak29399de787644 Black Lace-Up Sneakers', 'Green foonike-men-s-tshak29399de787644 High Top Suede Leather Sneakers', 'Green foonike-men-s-tshak29399de787644 Blue Solid Sneakers', 'Green foonike-men-s-tshak29399de787644 Grey Rubber Sneakers']
['Green foonike-men-s-tshak29399de787644 Running Shoes Mesh Sneakers', 'Grey nike Grey EVA Sneakers', 'Blue nike Blue 1 Pair Sneakers']
[]
[['https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/bestof2020-1589485703.jpg?crop=0.502xw:1.00xh;0.498xw,0&resize=640:*'], ['https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/index-2-1596820440.png?crop=0.497xw:0.993xh;0.252xw,0&resize=640:*'], [], ['https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/hikingshoes-1596638780.jpg?crop=0.502xw:1.00xh;0.250xw,0&resize=640:*'], ['https://

127.0.0.1 - - [09/Aug/2020 12:57:04] "POST / HTTP/1.1" 200 -


[['ORGANIZATION/ORGANIZATION_TYPE', 'BUSINESS/INDUSTRY', 'BUSINESS/PRODUCT_CATEGORY', 'BUSINESS/PRODUCT_LINE', 'LOCATION/COUNTRY', 'ORGANIZATION/CLUB_INTEREST', 'LOCATION/CITYTOWN', 'BUSINESS/SPONSOR', 'ORGANIZATION/ORGANIZATION', 'CELEBRITIES/CELEBRITY', 'MUSIC/PRODUCER', 'FILM/PERSON_OR_ENTITY_APPEARING_IN_FILM', 'FASHION/FASHION_DESIGNER', 'INTERNET/SOCIAL_NETWORK_USER', 'TV/TV_PERSONALITY', 'MUSIC/ARTIST'], ['BUSINESS/PRODUCT_CATEGORY', 'BUSINESS/PRODUCT_LINE', 'BUSINESS/INDUSTRY', 'LOCATION/COUNTRY', 'BUSINESS/SPONSOR', 'ORGANIZATION/ORGANIZATION', 'LOCATION/CITYTOWN', 'ORGANIZATION/ORGANIZATION_TYPE', 'MUSIC/PRODUCER', 'MUSIC/ARTIST'], ['BUSINESS/SPONSOR', 'ORGANIZATION/ORGANIZATION', 'BUSINESS/PRODUCT_CATEGORY', 'BUSINESS/PRODUCT_LINE', 'BUSINESS/INDUSTRY', 'MUSIC/GENRE', 'MUSIC/ARTIST', 'LOCATION/REGION', 'LOCATION/CITYTOWN', 'MUSIC/PRODUCER', 'FILM/PERSON_OR_ENTITY_APPEARING_IN_FILM', 'CELEBRITIES/CELEBRITY', 'INTERNET/SOCIAL_NETWORK_USER'], ['LOCATION/CITYTOWN', 'BUSINESS/IND

127.0.0.1 - - [09/Aug/2020 12:58:12] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2020 12:58:23] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2020 12:58:34] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2020 12:58:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2020 12:59:18] "GET / HTTP/1.1" 200 -
